I need to able to dynamically change the number of parameters, but writing the kernel code for different amounts of parameters sucks so I need some sort of automated way of doing this.

In [8]:
import math

In [56]:
# An atomic term is a bunch of variables multiplied together
class atomic_term:

    def __init__(self, input):
        self.components = input

    def __str__(self):

        out = ""
        
        for comp in self.components:
            out += str(comp)
            out += "*"

        return out[0:-1]


    # Gets variables 
    def variables(self):
        return [v for v in self.components if (isinstance(v, str) and v != "i")]
    
    
    # Gets variables and i's
    def nonConstants(self):
        return [v for v in self.components if isinstance(v, str)]
    
    # Gets the product of all constants
    def constant(self):
        return math.prod([c for c in self.components if isinstance(c, int)])
    
    
    # Checks if this term and input term have the same list of variables
    def like(self, inpTerm):

        # Get sorted list of non constants
        selfVars = sorted(self.nonConstants())
        inpVars = sorted(inpTerm.nonConstants())

        if len(selfVars) != len(inpVars):
            return False
        

        for i in range(len(selfVars)):
            if selfVars[i] != inpVars[i]:
                return False
            
        return True

    # Combines all i's and constant terms then sorts
    def clean(self):
        
        # New list of components to be filled
        newComponents = []

        # Get all constants and multiply them
        newConstant = self.constant()

        # If newConstant is zero make the entire term 0
        if newConstant == 0:
            self.components = [0]
            return

        # Count the number of i's
        iCount = len([im for im in self.components if im == "i"])

        # Multiply the constants with the i's
        if (iCount % 4 == 2 or iCount % 4 == 3):
            newConstant = -1 * newConstant 

        # Get the variables and sort them
        variables = [v for v in self.components if (isinstance(v, str) and v != "i")]
        variables.sort()

        # Add the variables into newComponents
        newComponents = variables
        
        # Add in i if necessary
        if (iCount%4 == 1 or iCount%4 == 3):
            newComponents.insert(0, "i")

        # Add the combined constants, if 1 and new components is not empty then ignore
        if (not newConstant == 1) or (len(newComponents) == 0):
            newComponents.insert(0, newConstant)

        self.components = newComponents

        

        

# A term sum is a sum of atomic terms
class term_sum:

    def __init__(self, terms):
        self.components = terms

    def __str__(self):
        out = ""

        for term in self.components:
            out += term.__str__()
            out += " + "
        
        return out[0:-3]

    def sort(self):
        self.components.sort()

    def clean(self):

        for term in self.components:
            term.clean()

        # Remove all zero terms
        zeroTerms = [term for term in self.components if len(term.components) == 1 and term.components[0] == 0]
        for term in zeroTerms:
            self.components.remove(term)

    # Combines terms which only differ by a (real) constant
    # Keeps terms which differ by i separate
    def combine_like_terms(self):

        # Necessary to combine all constants into one and account for i's
        self.clean()

        # List to be filled with new atomic terms
        newAtomicTerms = []

        # Repeatedly take this objects first term and combine it with all similar terms
        while len(self.components) != 0:
                
            # Find all atomic terms which have the same list of variables as the first term
            likeTerms = [term for term in self.components if self.components[0].like(term)]

            # Delte all these terms from self
            for term in likeTerms: self.components.remove(term)

            # Combine these terms into one and add to newAtomicTerms
            newConstant = sum([term.constant() for term in likeTerms])
            newAtomicTerms.append(atomic_term([newConstant] + likeTerms[0].nonConstants()))


        # Set the components of this object to be newAtomicTerms
        self.components = newAtomicTerms





# Multiplies two atomic terms, just combines the component lists. Does not clean.
def mulitply_atomic(term1, term2):
    return atomic_term(term1.components + term2.components)



# Multiplies two term_sum's. Returns a term sum. Does not clean.
def multiply_out_terms(terms1, terms2):

    outTermSum = term_sum([])

    for term1 in terms1.components:
        for term2 in terms2.components:
            outTermSum.components.append(mulitply_atomic(term1, term2))

    return outTermSum



# Puts term_sum in format that can be put in code. Only enter in clean terms for convenience. 
# Does not separate real and imaginary parts, this must be done beforehand.
# Does not factor out variables in the nonFactor list
def recursive_format_term_sum(inpTermSum, noFactor):

    # Find the counts of each variable
    variableCounts = helper_variable_counts(inpTermSum, noFactor)


    # Base case: all terms are in nofactor so there is nothing to factor
    if len(variableCounts.values()) == 0:
        return inpTermSum.__str__()


    # Base case: No variable has a count over 1
    # This means that there are no terms that we could factor out to improve efficiency
    # If a term only appears once, factoring it out would turn
    # one multiplication into one multiplication
    if max(variableCounts.values()) <= 1:
        return inpTermSum.__str__()


    # Find the variable with the highest count
    # This is the one we will factor out
    variableToFactor = sorted(variableCounts.items(), key=lambda item: item[1], reverse=True)[0][0]


    # Factor out one copy of variableToFactor from each atomic term in inpTermSum
    nonFactoredTerms = term_sum([])
    factoredTerms = term_sum([])

    for atomicTerm in inpTermSum.components:

        if variableToFactor in atomicTerm.components:
            termCopy = atomicTerm.components.copy()
            termCopy.remove(variableToFactor)
            factoredTerms.components.append(
                atomic_term(
                    termCopy
                )
            )
        else:
            nonFactoredTerms.components.append(
                atomic_term(
                    atomicTerm.components.copy()
                )
            )


    # Check for empty nonFactoredTerms
    if len(nonFactoredTerms.components) == 0:
        return "(" + recursive_format_term_sum(factoredTerms, noFactor) + ") * " + variableToFactor
    else:
        return recursive_format_term_sum(nonFactoredTerms, noFactor) + " + (" + recursive_format_term_sum(factoredTerms, noFactor) + ") * " + variableToFactor



# Finds the counts of each variable
# Each term containing one or more of a variables only counts as one instance of the variable
# Ignore the variables in noFactor
# e.x. a*a + a*b + c*2 counts 2 a's, 1 b, and 1 c
def helper_variable_counts(inpTermSum, noFactor):
    
    variableSet = set()                             # Each unique variable
    for atomicTerm in inpTermSum.components:
        for term in atomicTerm.components:
            if (not isinstance(term, int)) and term not in noFactor:
                variableSet.add(term)

    variableDict = dict()                           # Stores counts of each variable
    for variable in variableSet:
        variableDict[variable] = 0


    # Count each distinct appearance of a variable 
    for atomicTerm in inpTermSum.components:

        for variable in variableSet:

            if variable in atomicTerm.components:
                variableDict[variable] += 1

    return variableDict


# Gets the imaginary and real components of a term_sum and returns them as term_sum's
def separate_parts(inpTermSum):

    inpTermSum.clean()

    realTerms = []
    imaginaryTerms = []

    for atomicTerm in inpTermSum.components:

        if "i" in atomicTerm.components:
            imaginaryTerms.append(atomic_term([var for var in atomicTerm.components if var != "i"]))
        else:
            realTerms.append(atomic_term([var for var in atomicTerm.components]))

    return (term_sum(realTerms), term_sum(imaginaryTerms))


def code_format(inpTermSum, noFactor):

    inpTermSum.clean()

    realPart, imagPart = separate_parts(inpTermSum)

    realPart.clean()
    print(recursive_format_term_sum(realPart, noFactor))

    imagPart.clean()
    print(recursive_format_term_sum(imagPart, noFactor))



In [69]:
buildTerm = term_sum([atomic_term([1])])

buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["zr"]),atomic_term([-1, "i","zi"])]))

print(buildTerm)



1*zr + 1*-1*i*zi


In [40]:
atomic_term([1]).like(atomic_term([1]))

True

In [73]:
# Build up a large term_sum that is the polynomial I need for the 52 parameter kernel
kernel_term_sum = term_sum([])

# Iterate through the different exponents for c, z, and zb
for cExp in range(3):
    for zExp in range(3):
        for zbExp in range(3):

            # Ignore the all zeros case
            if cExp + zExp + zbExp != 0:
                
                # Build up this term_sum by multiplying the correct amount of c', z's and zb's
                buildTerm = term_sum([atomic_term([1])])

                for i in range(cExp):
                    buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["cr"]),atomic_term(["i","ci"])])) # *(cr + i*ci)

                for i in range(zExp):
                    buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["zr"]),atomic_term(["i","zi"])])) # *(zr + i*zi)

                for i in range(zbExp):
                    buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["zr"]),atomic_term([-1, "i","zi"])])) # *(zr - i*zi)

                # Add the coefficient
                buildTerm = multiply_out_terms(buildTerm, term_sum([atomic_term(["coeffs[" + str(cExp + 3*zExp + 9*zbExp - 1) + "]"])]))

                # Add the components of buildTerm to kernel_term_sum
                for atomicTerm in buildTerm.components:
                    kernel_term_sum.components.append(atomic_term(atomicTerm.components))


kernel_term_sum.combine_like_terms()
kernel_term_sum.clean()

code_format(kernel_term_sum, ["cr", "ci"])




coeffs[0]*cr + (cr*cr + -1*ci*ci) * coeffs[1] + (ci*coeffs[9] + -1*ci*coeffs[3] + 2*ci*coeffs[10]*cr + -2*ci*coeffs[4]*cr + (-1*coeffs[17] + coeffs[11] + -1*coeffs[5] + -1*coeffs[18]*cr + coeffs[12]*cr + -1*coeffs[6]*cr + (-1*cr*cr + ci*ci) * coeffs[7] + (-1*cr*cr + ci*ci) * coeffs[19] + (cr*cr + -1*ci*ci) * coeffs[13] + (ci*coeffs[21] + -1*ci*coeffs[15] + 2*ci*coeffs[22]*cr + -2*ci*coeffs[16]*cr + (coeffs[23] + coeffs[24]*cr + (cr*cr + -1*ci*ci) * coeffs[25]) * zi) * zi) * zi) * zi + (coeffs[8] + coeffs[2] + coeffs[9]*cr + coeffs[3]*cr + (cr*cr + -1*ci*ci) * coeffs[4] + (cr*cr + -1*ci*ci) * coeffs[10] + (2*ci*coeffs[18] + -2*ci*coeffs[6] + 4*ci*coeffs[19]*cr + -4*ci*coeffs[7]*cr + (coeffs[20] + coeffs[14] + coeffs[21]*cr + coeffs[15]*cr + (cr*cr + -1*ci*ci) * coeffs[16] + (cr*cr + -1*ci*ci) * coeffs[22]) * zi) * zi + (coeffs[17] + coeffs[11] + coeffs[5] + coeffs[18]*cr + coeffs[12]*cr + coeffs[6]*cr + (cr*cr + -1*ci*ci) * coeffs[7] + (cr*cr + -1*ci*ci) * coeffs[19] + (cr*cr + -1*ci*ci

In [143]:
# Some string processing things to allow for the code to be optimized

# Finds matching ( to a )
def matchingParentheses(code, index):

    if code[index] != ")":
        raise ValueError("Not a ) !!")
    
    iterIndex = index - 1
    parenCount = 1
    
    while iterIndex >= 0 and parenCount != 0:

        if code[iterIndex] == ")": parenCount += 1

        if code[iterIndex] == "(": parenCount += -1

        iterIndex += -1

    if iterIndex < 0:
        raise TypeError("No matching (")
    
    return iterIndex + 1



replaceCount = 0
replacements = []

def swap_constants(code):

    global replaceCount
    global replacements

    replaceCount = 0
    replacements = []

    return swap_constants_recursive(code)



# Recursively switch out contant terms for parameters
# Returns new code and list of parametters replaced
def swap_constants_recursive(code):

    global replaceCount
    global replacements

    # Base case: No z's in code
    # No need to do any recursion, just replace with
    if "z" not in code:
        replaceCount += 1
        replacements.append("float p" + str(replaceCount) + " = " + code + ";")
        return "p" + str(replaceCount)
    
    # Recursive case: z exists in code

    # 1. Find the first instance of z from the right
    zIndex = len(code) - 1
    while zIndex >= 0 and code[zIndex] != "z":
        zIndex += -1

    # 2. Locate the ")" closest to this z and its corresponding "("
    paren1Index = zIndex - 4
    paren2Index = matchingParentheses(code, paren1Index)

    # 3. Find the string multiplying z and the string added on the side
    multString = code[paren2Index+1:paren1Index]
    addString = code[0:paren2Index - 3]

    # 4. Call swap_constants_recursive 
    return swap_constants_recursive(addString) + " + (" + swap_constants_recursive(multString) + ")" + code[paren1Index+1:]




code = "coeffs[0]*cr + (cr*cr + -1*ci*ci) * coeffs[1] + (ci*coeffs[9] + -1*ci*coeffs[3] + 2*ci*coeffs[10]*cr + -2*ci*coeffs[4]*cr + (-1*coeffs[17] + coeffs[11] + -1*coeffs[5] + -1*coeffs[18]*cr + coeffs[12]*cr + -1*coeffs[6]*cr + (-1*cr*cr + ci*ci) * coeffs[7] + (-1*cr*cr + ci*ci) * coeffs[19] + (cr*cr + -1*ci*ci) * coeffs[13] + (ci*coeffs[21] + -1*ci*coeffs[15] + 2*ci*coeffs[22]*cr + -2*ci*coeffs[16]*cr + (coeffs[23] + coeffs[24]*cr + (cr*cr + -1*ci*ci) * coeffs[25]) * zi) * zi) * zi) * zi + (coeffs[8] + coeffs[2] + coeffs[9]*cr + coeffs[3]*cr + (cr*cr + -1*ci*ci) * coeffs[4] + (cr*cr + -1*ci*ci) * coeffs[10] + (2*ci*coeffs[18] + -2*ci*coeffs[6] + 4*ci*coeffs[19]*cr + -4*ci*coeffs[7]*cr + (coeffs[20] + coeffs[14] + coeffs[21]*cr + coeffs[15]*cr + (cr*cr + -1*ci*ci) * coeffs[16] + (cr*cr + -1*ci*ci) * coeffs[22]) * zi) * zi + (coeffs[17] + coeffs[11] + coeffs[5] + coeffs[18]*cr + coeffs[12]*cr + coeffs[6]*cr + (cr*cr + -1*ci*ci) * coeffs[7] + (cr*cr + -1*ci*ci) * coeffs[19] + (cr*cr + -1*ci*ci) * coeffs[13] + (ci*coeffs[21] + -1*ci*coeffs[15] + 2*ci*coeffs[22]*cr + -2*ci*coeffs[16]*cr + (2*coeffs[23] + 2*coeffs[24]*cr + (2*cr*cr + -2*ci*ci) * coeffs[25]) * zi) * zi + (coeffs[20] + coeffs[14] + coeffs[21]*cr + coeffs[15]*cr + (cr*cr + -1*ci*ci) * coeffs[16] + (cr*cr + -1*ci*ci) * coeffs[22] + (coeffs[23] + coeffs[24]*cr + (cr*cr + -1*ci*ci) * coeffs[25]) * zr) * zr) * zr) * zr"



In [144]:

print(code)
swapped = swap_constants(code)
print(swapped)
print()
print(replacements)


coeffs[0]*cr + (cr*cr + -1*ci*ci) * coeffs[1] + (ci*coeffs[9] + -1*ci*coeffs[3] + 2*ci*coeffs[10]*cr + -2*ci*coeffs[4]*cr + (-1*coeffs[17] + coeffs[11] + -1*coeffs[5] + -1*coeffs[18]*cr + coeffs[12]*cr + -1*coeffs[6]*cr + (-1*cr*cr + ci*ci) * coeffs[7] + (-1*cr*cr + ci*ci) * coeffs[19] + (cr*cr + -1*ci*ci) * coeffs[13] + (ci*coeffs[21] + -1*ci*coeffs[15] + 2*ci*coeffs[22]*cr + -2*ci*coeffs[16]*cr + (coeffs[23] + coeffs[24]*cr + (cr*cr + -1*ci*ci) * coeffs[25]) * zi) * zi) * zi) * zi + (coeffs[8] + coeffs[2] + coeffs[9]*cr + coeffs[3]*cr + (cr*cr + -1*ci*ci) * coeffs[4] + (cr*cr + -1*ci*ci) * coeffs[10] + (2*ci*coeffs[18] + -2*ci*coeffs[6] + 4*ci*coeffs[19]*cr + -4*ci*coeffs[7]*cr + (coeffs[20] + coeffs[14] + coeffs[21]*cr + coeffs[15]*cr + (cr*cr + -1*ci*ci) * coeffs[16] + (cr*cr + -1*ci*ci) * coeffs[22]) * zi) * zi + (coeffs[17] + coeffs[11] + coeffs[5] + coeffs[18]*cr + coeffs[12]*cr + coeffs[6]*cr + (cr*cr + -1*ci*ci) * coeffs[7] + (cr*cr + -1*ci*ci) * coeffs[19] + (cr*cr + -1*ci*ci

In [140]:
# 1. Find the first instance of z from the right
zIndex = len(code) - 1
while zIndex >= 0 and code[zIndex] != "z":
    zIndex += -1

# 2. Locate the ")" closest to this z and its corresponding "("
paren1Index = zIndex - 4
paren2Index = matchingParentheses(code, paren1Index)


code[paren2Index+1:paren1Index]

'coeffs[8] + coeffs[2] + coeffs[9]*cr + coeffs[3]*cr + (cr*cr + -1*ci*ci) * coeffs[4] + (cr*cr + -1*ci*ci) * coeffs[10] + (2*ci*coeffs[18] + -2*ci*coeffs[6] + 4*ci*coeffs[19]*cr + -4*ci*coeffs[7]*cr + (coeffs[20] + coeffs[14] + coeffs[21]*cr + coeffs[15]*cr + (cr*cr + -1*ci*ci) * coeffs[16] + (cr*cr + -1*ci*ci) * coeffs[22]) * zi) * zi + (coeffs[17] + coeffs[11] + coeffs[5] + coeffs[18]*cr + coeffs[12]*cr + coeffs[6]*cr + (cr*cr + -1*ci*ci) * coeffs[7] + (cr*cr + -1*ci*ci) * coeffs[19] + (cr*cr + -1*ci*ci) * coeffs[13] + (ci*coeffs[21] + -1*ci*coeffs[15] + 2*ci*coeffs[22]*cr + -2*ci*coeffs[16]*cr + (2*coeffs[23] + 2*coeffs[24]*cr + (2*cr*cr + -2*ci*ci) * coeffs[25]) * zi) * zi + (coeffs[20] + coeffs[14] + coeffs[21]*cr + coeffs[15]*cr + (cr*cr + -1*ci*ci) * coeffs[16] + (cr*cr + -1*ci*ci) * coeffs[22] + (coeffs[23] + coeffs[24]*cr + (cr*cr + -1*ci*ci) * coeffs[25]) * zr) * zr) * zr'